In [10]:
using Plots
#gr()
pyplot()
#plotly()

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/StaticArrays.ji for module StaticArrays.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/RecipesBase.ji for module RecipesBase.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/ColorTypes.ji for module ColorTypes.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/Showoff.ji for module Showoff.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/NaNMath.ji for module NaNMath.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/Measures.ji for module Measures.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/PyPlot.ji for module PyPlot.


Plots.PyPlotBackend()

## A. Data Table choices

Machine learning is all about finding patterns in data, so it is very reasonble to start with data.

In [11]:
# Some Data (try your own)
x = [5,6.5,7,8]
y = [10.1, 19.9, 30.1, 40.3]
# plot(x,y,
#     label="Y", line=(7,:green), marker=(10,0.8,:red), xlims=(0,10), ylims=(0,50),
#     xlabel="X",ylabel="Y")
    

4-element Array{Float64,1}:
 10.1
 19.9
 30.1
 40.3

### A.1. Just a matrix please. (No labels, no extras, simple.)

In [12]:
data1 = [x y]

4×2 Array{Float64,2}:
 5.0  10.1
 6.5  19.9
 7.0  30.1
 8.0  40.3

### A.2. Data Frames: Inspired by the R universe.

In [13]:
using DataFrames
data2 = DataFrame(X=x,Y=y) # Upper Case X and Y are labels (not data)

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/DataFrames.ji for module DataFrames.


,X,Y
1,5.0,10.1
2,6.5,19.9
3,7.0,30.1
4,8.0,40.3


In [14]:
data2[1]

4-element DataArrays.DataArray{Float64,1}:
 5.0
 6.5
 7.0
 8.0

In [15]:
#Pkg.add("CSV")
using CSV
CSV.write("data.csv", data2)

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/NullableArrays.ji for module NullableArrays.
INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/CSV.ji for module CSV.
  likely near /Users/edelman/.julia/v0.6/IJulia/src/kernel.jl:31
  likely near /Users/edelman/.julia/v0.6/IJulia/src/kernel.jl:31


CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd", IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "data.csv", 8, true, String["X", "Y"], false)

In [16]:
;cat data.csv

"X","Y"
5.0,10.1
6.5,19.9
7.0,30.1
8.0,40.3


### A.3. Indexed Tables (Treat data like array indices, knows type information)

In [17]:
# Pkg.add("IndexedTables")
using  IndexedTables.Table
using IndexedTables
data3 = Table(Columns(X=x),Columns(Y=y))

X   │ Y
────┼─────
5.0 │ 10.1
6.5 │ 19.9
7.0 │ 30.1
8.0 │ 40.3

In [18]:
data3[6.5]

(Y = 19.9)

In [19]:
typeof.([data1,data2,data3])

3-element Array{DataType,1}:
 Array{Float64,2}                                                                                                                                                                                                                                 
 DataFrames.DataFrame                                                                                                                                                                                                                             
 IndexedTables.IndexedTable{NamedTuples._NT_Y{Float64},Tuple{Float64},IndexedTables.Columns{NamedTuples._NT_X{Float64},NamedTuples._NT_X{Array{Float64,1}}},IndexedTables.Columns{NamedTuples._NT_Y{Float64},NamedTuples._NT_Y{Array{Float64,1}}}}

### A.4. JuliaDB (Lots of bells and whistles, many files, parallelism, ...)

In [ ]:
#Pkg.add("JuliaDB")
using JuliaDB:DTable
using JuliaDB

INFO: Precompiling module JuliaDB.
This may mean module IndexedTables does not support precompilation but is imported by a module that does.
ERROR: LoadError: Declaring __precompile__(false) is not allowed in files that are being precompiled.
Stacktrace:
 [1] 

In [21]:
data4 = distribute(data3, 1) 

LoadError: [91mUndefVarError: distribute not defined[39m

In [ ]:
data5 = loadfiles(["data.csv"], usecache=false)

In [ ]:
typeof(data4)

In [ ]:
data4[1:2]

In [ ]:
select(data4,1=>i->i≥7) 

In [ ]:
filter(t->(t[1]>30),data4) 

### A.5 IterableTables

In [ ]:

#using IterableTables, DataTables, TypedTables # haven't investigated  much but looks very nice

## B. Simple Line Fitting

[So why is it called "Regression" anyway?](http://blog.minitab.com/blog/statistics-and-quality-data-analysis/so-why-is-it-called-regression-anyway) Dalton's original meaning not quite what it means today.

B.1 Linear Regression function

In [4]:
b, w =  linreg(x,y)

(-42.45733333333333, 10.197333333333333)

In [ ]:
plot()
plot(x,y,
    label="Y", line=(4,:blue), marker=(3,0.8,:blue), xlims=(0,10), ylims=(0,50),
    xlabel="X",ylabel="Y")
plot!(x->w*x+b,xlims=(minimum(x)-.5,maximum(x)+.5), line=(4,:red), label="best fit line")
plot!(x->w*x+b, x ,marker=(3,0.8,:red), label="" )
for i = 1:length(x)
    plot!([x[i],x[i]],[y[i],w*x[i]+b],line=(4,:green))
end
plot!(legend=:topleft)

Mathematically equivalent Approaches <br>
B.2 Linear Algebra Least Squares

In [ ]:
A = [ones(x) x]

In [ ]:
A'A

In [ ]:
A\y 

In [ ]:
(A'A)\A'y  # normal equations usually not recommended

In [ ]:
q,r = qr(A)
r\(q'y)

In [ ]:
[length(x) sum(x); sum(x) x⋅x] \ [ sum(y) ; x⋅y ] # (A'A)\A'y

B.3 Basic Formula

In [ ]:
w = cov(x,y)/var(x) # same as (x.-mean(x))⋅(y.-mean(y))/sum(abs2,x.-mean(x))
b = mean(y)-w*mean(x)
b,w

In [ ]:
@which linreg(x,y) # essentially uses the above formula

B.4 optimization  (think machine learning) via the package optim.jl

In [ ]:
using Optim   # Julia all the way down
loss(bw) = sum(abs2,bw[2]*x.+bw[1]-y) # uglyish
optimize(loss,[0.0,0.0]).minimizer

B.5 optimization with the package JuMP <br>
Note not every julia function can be in @objective or @NLobjective
but that would be the goal. See  [linear and quadratic objective Jump Notes](http://www.juliaopt.org/JuMP.jl/0.18/refexpr.html)  and [Nonlinear Jump Notes](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html#syntax-notes).

In [ ]:
#Pkg.add("Ipopt")

In [ ]:
using JuMP, Ipopt
n = length(x)
m = Model(solver=IpoptSolver(print_level=0))
@variable(m,w)
@variable(m,b)
@objective(m, Min, sum((w*x[i]+b-y[i])^2 for i in 1:n))
#@objective(m, Min,   sum(abs2,  w*x+b-y))
solve(m)
println( " b = ", getvalue(b), "w = ", getvalue(w))

B.6 Generalized Linear Models <br>
the very fancy statistical thing

In [ ]:
#Pkg.add("GLM")
using GLM # Generalized Linear Models

In [ ]:
lm(@formula(Y~X), data2)

The lines above are obviously b and w
We assume at the start X is known without error, b,w,σ are unknown and
the real Y is distributed like  b+w*X+$\sigma *$randn(),
and the Y we have are samples from this distribution.

Under these assumptions, if we fit many times, the b and w would be normal, with these predicted standard deviations.

The third column is just the ratio of column 1 to column 2 , thus normalizing the situation to a standard normal.

When the probability column is less than .05, we can reject the hypothesis that the intercept/slope is 0 at the 5 percent signficance level. What does this mean? It means we feel pretty good about our intercept and slope. If the probability is higher than .05 we can not reject the null hypothesis, meaning that we feel 0 for the intercept/slope could have been possible. In particular a 0 slope says that the dependent variable is not really statistically dependent after all.

### C. Stochastic Gradient Descent

In [ ]:
loss(w,b,i) =(w*x[i]+b-y[i])^2  # loss due to point i
Dloss(w,b,i) = 2*(w*x[i]+b-y[i])*[x[i];1]

In [ ]:
w,b = 0.0, 0.0
for t=1:100000
    η = .002  # there seems to be an art to picking these steplengths
    i = rand(1:4)
    d = Dloss(w,b,i)
    w -= η * d[1]
    b -= η * d[2]  
end
 println(b," ",w)   

In [ ]:
loss(w,b,i) =(w*χ[i]+b-y[i])^2  # loss due to point i
Dloss(w,b,i) = 2*(w*χ[i]+b-y[i])*[χ[i];1]

In [ ]:
μ = mean(x)
σ = std(x)
χ = (x-μ)/σ

w,b = 0.0, 0.0
for t=1:100000
    η = .01  # there seems to be an art to picking these steplengths
    i = rand(1:4)
    d = Dloss(w,b,i)
     w -= η * d[1]
     b -= η * d[2] 
    ## instead fancy update rules like Adam ??
   
end
 println(b-w*μ/σ," ",w/σ)


###  D. KNET

In [ ]:
#Pkg.add("Knet")
using Knet

In [ ]:
predict(w,x) = w[2]*x .+ w[1]
loss(w,x,y) = sum(abs2, y - predict(w,x)) 

In [ ]:
lossgradient = grad(loss)

In [ ]:
function train(w, data; lr=.1)
    p=1
    for (x,y) in data
        println("This is pass $p")
        p+=1
        dw = lossgradient(w, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end
    end
    return w
end

In [ ]:
train([0.0,0.0],zip(x,y),lr=.01) # not enough data

In [ ]:
data = [(x[i],y[i]) for i=1:4]

In [ ]:
function train2(w, data; lr=.1)
       for t in 1:10000
          
        (x,y) = data[rand(1:4)]
        dw = lossgradient(w, x, y)
            for i=1:length(w)
            w[i] -= lr * dw[i]
            #update(w, lossgradient(w,x,y), adam())
        end
    end
    return w
end

In [ ]:
train2([0.0;0.0],data,lr=.01) 

### E. TensorFlow

In [1]:
#Pkg.add("TensorFlow")
using TensorFlow
session = Session()

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/TensorFlow.ji for module TensorFlow.
2017-10-10 07:22:53.779166: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-10-10 07:22:53.779194: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


Session(Ptr{Void} @0x00000001211cee50)

In [2]:
W = TensorFlow.Variable(randn())
b = TensorFlow.Variable(randn())

TensorFlow.Variables.Variable{Float64}(<Tensor node_2:1 shape=() dtype=Float64>, <Tensor node_2/Assign:1 shape=unknown dtype=Float64>)

In [3]:
@tf X = placeholder(Float32)
@tf Y = multiply(X,W).+b
@tf Y_obs = placeholder(Float32)

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##712#713)(::TensorFlow.Tensor{Float64}, ::TensorFlow.Variables.Variable{Float64}) at ./deprecated.jl:346
 [3] (::TensorFlow.###8#9#11{Base.##712#713})(::Array{Any,1}, ::Function, ::TensorFlow.Tensor{Float64}, ::Vararg{Any,N} where N) at /Users/edelman/.julia/v0.6/TensorFlow/src/meta.jl:67
 [4] (::TensorFlow.##8#10)(::TensorFlow.Tensor{Float64}, ::Vararg{Any,N} where N) at /Users/edelman/.julia/v0.6/TensorFlow/src/meta.jl:67
 [5] include_string(::String, ::String) at ./loading.jl:515
 [6] include_string(::Module, ::String, ::String) at /Users/edelman/.julia/v0.6/Compat/src/Compat.jl:464
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/edelman/.julia/v0.6/IJulia/src/execute_request.jl:154
 [8] eventloop(::ZMQ.Socket) at /Users/edelman/.julia/v0.6/IJulia/src/eventloop.jl:8
 [9] (::IJulia.##14#17)() at ./task.jl:335
while loading In[3], in expression starting on line 2


<Tensor Y_obs:1 shape=unknown dtype=Float32>

In [4]:
@tf Loss=sum( (Y.-Y_obs).^2 )

<Tensor reduce:1 shape=unknown dtype=Float64>

In [5]:
optimizer = TensorFlow.train.GradientDescentOptimizer(1e-3)
minimizer = TensorFlow.train.minimize(optimizer, Loss)

<Tensor Group:1 shape=unknown dtype=Any>

In [6]:
run(session, global_variables_initializer())
for i in 1:20000
    run(session, minimizer, Dict(X=>x, Y_obs=>y))
end

LoadError: [91mUndefVarError: x not defined[39m

In [7]:
run(session, [b, W])

2-element Array{Float64,1}:
 -1.32189 
  0.907986

In [8]:
visualize()

In [ ]:
#####

In [ ]:
using TensorFlow

In [ ]:
session=Session()

In [ ]:
@tf X=placeholder(Float32)

In [ ]:
@tf W=get_variable([], Float32)

In [ ]:

@tf b=get_variable([], Float32)

In [ ]:
@tf Y=X.*W + b

In [ ]:
@tf Y_obs=placeholder(Float32)

In [ ]:
@tf Loss=reduce_sum((Y.-Y_obs).^2)

In [ ]:
optimizer=train.GradientDescentOptimizer(1e-3)

In [ ]:
minimizer=train.minimize(optimizer, Loss)

In [ ]:
run(session, global_variables_initializer())
for i in 1:20000
    run(session, minimizer, Dict(X=>[5,6.5,7,8], Y_obs=>[10.1,19.9,30.1,40.3]))
end

In [ ]:
run(session, [b, W])

In [ ]:
visualize()

In [13]:
Pkg.update()

INFO: Updating METADATA...
INFO: Updating cache of Plots...
INFO: Updating cache of Lazy...
INFO: Updating cache of StatPlots...
INFO: Updating cache of CategoricalArrays...
INFO: Updating cache of NNlib...
INFO: Updating cache of IJulia...
INFO: Updating cache of JuliaDB...
INFO: Updating cache of FFTW...
INFO: Updating cache of WeakRefStrings...
INFO: Updating cache of DataStreams...
INFO: Updating cache of RecipesBase...
INFO: Updating cache of Compat...
INFO: Updating cache of TensorFlow...
INFO: Updating cache of ShowItLikeYouBuildIt...
INFO: Updating cache of Flux...
INFO: Updating cache of Parameters...
INFO: Updating cache of Codecs...
INFO: Updating cache of OffsetArrays...
INFO: Updating cache of SpecialFunctions...
INFO: Updating cache of IndexedTables...
INFO: Updating Interact master...
INFO: Package SpecialFunctions: skipping update (pinned)...
INFO: Computing changes...
INFO: Upgrading Codecs: v0.3.0 => v0.4.0
INFO: Upgrading Compat: v0.32.0 => v0.33.0
INFO: Upgrading Fl